# Question 2

### Conversion between quaternions and Euler angles

References: 
- https://en.wikipedia.org/wiki/Conversion_between_quaternions_and_Euler_angles
- https://www.anyleaf.org/blog/quaternions:-a-practical-guide
- https://automaticaddison.com/how-to-convert-a-quaternion-into-euler-angles-in-python/

Concepts:
- Spatial rotations in three dimensions can be parametrized using both Euler angles and unit quaternions
- A quaternion has 4 scalar values: qw (the real part) and qx qy qz (the imaginary part)

In [7]:
# Convert quaternion (qx, qy, qz, qw) to Euler angles (yaw, pitch, roll) in degrees
import math
from typing import List

class Quaternion:
    def __init__(self, qlist: List[float]):
        """Init with [w, x, y, z]"""
        self.w = qlist[0]
        self.x = qlist[1]
        self.y = qlist[2]
        self.z = qlist[3]

    def inverse(self) -> List[float]:
        return [self.w, -self.x, -self.y, -self.z]


In [33]:
# init quaternion
q = Quaternion([1, 2, 0, -1])
print(q.inverse())
q_inv = Quaternion(q.inverse())

[1, -2, 0, 1]


#### Convert to Roll

In [40]:
def quaternion_to_roll(q: Quaternion):
    """Roll: x-axis rotation"""
    sinr_cosp = 2.0 * (q.w * q.x + q.y * q.z)
    print(sinr_cosp)
    cosr_cosp = 1.0 - 2.0 * (q.x ** 2 + q.y ** 2)
    print(cosr_cosp)
    roll = math.atan2(sinr_cosp, cosr_cosp)
    return roll

roll = quaternion_to_roll(q)
print("roll =", roll)
print("roll degree =", math.degrees(roll))


4.0
-7.0
roll = 2.62244653934327
roll degree = 150.25511870305778


#### Convert to Yaw

In [38]:
def quaternion_to_yaw(q: Quaternion):
    """Yaw: z-axis rotation"""
    siny_cosp = 2.0 * (q.w * q.z + q.x * q.y)
    print(siny_cosp)
    cosy_cosp = 1.0 - 2.0 * (q.y ** 2 + q.z ** 2)
    print(cosy_cosp)
    yaw = math.atan2(siny_cosp, cosy_cosp)
    return yaw

yaw = quaternion_to_yaw(q)
print("yaw =", yaw)
print("yaw degree =", math.degrees(yaw))

-2.0
-1.0
yaw = -2.0344439357957027
yaw degree = -116.56505117707799


#### Convert to Pitch

In [56]:
def quaternion_to_pitch_old(q: Quaternion) -> float:
    """Pitch: y-axis rotation"""
    sinp = 2.0 * (q.w * q.y - q.z * q.x)
    #print(sinp)
    pitch = 2.0 * math.atan2(math.sqrt(1+sinp), math.sqrt(1-sinp)) - math.pi / 2
    return pitch

print(quaternion_to_pitch_old(q))

ValueError: math domain error

Directly follow the equation errored when `1 - sinp < 0`; Try another method

In [47]:
def quaternion_to_pitch(q: Quaternion) -> float:
    """Pitch: y-axis rotation"""
    sinp = 2.0 * (q.w * q.y - q.z * q.x)
    sinp = 1.0 if sinp > 1.0 else sinp
    sinp = -1.0 if sinp < -1.0 else sinp
    pitch = math.asin(sinp)
    return pitch

pitch = quaternion_to_pitch(q)
print("pitch =", pitch)
print("pitch degree =", math.degrees(pitch))

pitch = 1.5707963267948966
pitch degree = 90.0


In [54]:
def quaternion_to_euler_angles(qlist: List[float]):

    q = Quaternion(qlist)
    print(q.x, q.y, q.z, q.w)
    roll = math.degrees(quaternion_to_roll(q))
    pitch = math.degrees(quaternion_to_pitch(q))
    yaw = math.degrees(quaternion_to_yaw(q))

    return (roll, pitch, yaw)

In [55]:
# Test cases
test_cases = [
    [1, 2, 0, -1],
    [1, -2, 0, 1],
    [0.7072, 0, 0, 0.7072],
]
for test_case in test_cases:
    print(test_case)
    print("(roll, pitch, yaw) = ", quaternion_to_euler_angles(test_case))


[1, 2, 0, -1]
2 0 -1 1
4.0
-7.0
-2.0
-1.0
(roll, pitch, yaw) =  (150.25511870305778, 90.0, -116.56505117707799)
[1, -2, 0, 1]
-2 0 1 1
-4.0
-7.0
2.0
-1.0
(roll, pitch, yaw) =  (-150.25511870305778, 90.0, 116.56505117707799)
[0.7072, 0, 0, 0.7072]
0 0 0.7072 0.7072
0.0
1.0
1.0002636800000002
-0.0002636800000002104
(roll, pitch, yaw) =  (0.0, 0.0, 90.01510376823046)
